# customer segmentation with K-Means algorithm

## 1- Introduction

In this project, I use a dataset from the **Kaggle website** and cluster bank credit card customers into different groups based on the **K-means model**. It has been tried to cluster customers into the clusters and groups introduced in the following study:  
https://www.mckinsey.com/~/media/mckinsey/dotcom/client_service/financial%20services/latest%20thinking/payments/mop19_new%20frontiers%20in%20credit%20card%20segmentation.ashx  

**Data Address**: 
https://www.kaggle.com/datasets/arjunbhasin2013/ccdata

In [1]:
#importing necessary libraries and packages
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
# magic word for producing visualizations in notebook
%matplotlib inline
# For Standardisation
from sklearn.preprocessing import StandardScaler
# For Hierarchical clustering
from scipy.cluster.hierarchy import dendrogram, linkage
# For K-means
from sklearn.cluster import KMeans
# For PCA
from sklearn.decomposition import PCA

## 2- loading data and doing some explorations

In [2]:
#importing data
data_seg=pd.read_csv ('Customer_Data (1).csv',index_col=0)

FileNotFoundError: [Errno 2] No such file or directory: 'Customer_Data (1).csv'

In [3]:
data_seg.head ()

NameError: name 'data_seg' is not defined

In [4]:
data_seg.describe()

NameError: name 'data_seg' is not defined

In [5]:
_=data_seg.hist(bins=30, figsize=(30, 24))

NameError: name 'data_seg' is not defined

From the above histogram, we can conclude that almost all the variables are very **skewed and not symmetrical**.

In [6]:
data_seg.corr()

NameError: name 'data_seg' is not defined

Exploring the correlation between the features is the first step to identifying similar customers and putting them together in groups which is the essence of segmentation. In the above correlation matrix we can see that **PURCHASES Variable** have a high correlation with PURCHASES_TRX and PAYMENTS variables. It seems quite logical that the more the number of purchases, the higher the probability of the total purchase amount. Also, people who make more purchases with credit cards try to make their payments regularly and therefore the PAYMENTS level will be higher. People who use a credit card in the form of cash advance will also have a lower purchase amount (negative correlation between PURCHASES and CASH_ADVANCE).

In [7]:
#Correlation heatmap
plt.figure(figsize=(12,9))
Vis=sns.heatmap(data_seg.corr(),annot=True, cmap='RdBu',vmin=-1,vmax=1)
Vis.set_yticklabels(Vis.get_yticklabels(),rotation=0,fontsize=10)
Vis.set_xticklabels(Vis.get_xticklabels(),rotation=90,fontsize=10)
plt.title ('Correlation Heatmap')
plt.show ()

NameError: name 'data_seg' is not defined

<Figure size 1200x900 with 0 Axes>